# Episode 16: Handmade Parsers

Calculators are the wave of the past...long live calculators!

To make matters even more old school, let's write a simple calculator language by hand!

Grammar:

```
statements : EMPTY | statement statements
statement  : (assign | expression) '\n'
assign     : ID '=' expression
expression : atom (op expression)?
atom       : ID | NUM
op         : '+' | '-'
```

## Step 1: Lexical Analysis

In [1]:
import typing

EMPTY = 0
NL = 1
ID = 2
EQ = 3
NUM = 4
PLUS = 5
MINUS = 6

Token = typing.Tuple[int, str]

def calc_lexer(source: str) -> typing.Iterator[Token]:
    index = 0
    while index < len(source):
        char = source[index]
        if char in ' \t':
            pass
        elif char == '\n':
            yield (NL, char)
        elif char.isalpha():
            result = char
            index += 1
            if index < len(source):
                char = source[index]
                while index < len(source) and char.isalpha():
                    result += char
                    index += 1
                    char = source[index]
            yield (ID, result)
            continue
        elif char == '=':
            yield (EQ, char)
        elif char.isdigit():
            result = char
            index += 1
            if index < len(source):
                char = source[index]
                while index < len(source) and char.isdigit():
                    result += char
                    index += 1
                    char = source[index]
            yield(NUM, result)
            continue
        elif char == '+':
            yield (PLUS, char)
        elif char == '-':
            yield (MINUS, char)
        else:
            raise SyntaxError(f'Unexpected character in input string: {char}')
        index += 1
    yield (EMPTY, '')

In [2]:
list(calc_lexer('A = 2\nB = 3\nA + B'))

[(2, 'A'),
 (3, '='),
 (4, '2'),
 (1, '\n'),
 (2, 'B'),
 (3, '='),
 (4, '3'),
 (1, '\n'),
 (2, 'A'),
 (5, '+'),
 (2, 'B'),
 (0, '')]

## Step 2: Syntactic Analysis

In [3]:
import itertools

class Tokens:
    def __init__(self, stream: typing.Iterator[Token]):
        self.lookahead = []
        self.stream = stream
        
    def peek(self):
        next_token = next(self.stream)
        self.lookahead.append(next_token)
        return self.lookahead[-1]
    
    def unpeek(self):
        self.lookahead.reverse()
        self.stream = itertools.chain(self.lookahead, self.stream)
        self.lookahead = []
        
    def next_token(self):
        result = None
        if len(self.lookahead) > 0:
            result = self.lookahead.pop()
        else:
            result = next(self.stream)
        return result        

class Tree(typing.NamedTuple):
    contents: typing.Union[str, Token]
    children: typing.List['Tree']

def parse_statements(tokens: Tokens) -> Tree:
    token = tokens.peek()
    if token[0] == EMPTY:
        result = Tree(token, [])
    else:
        tokens.unpeek()
        child_0 = parse_statement(tokens)
        child_1 = parse_statements(tokens)
        result = Tree('statements', [child_0, child_1])
    return result

def parse_statement(tokens: Tokens) -> Tree:
    token = tokens.peek()
    if token[0] == ID:
        token = tokens.peek()
        tokens.unpeek()
        if token[0] == EQ:
            child_result = parse_assign(tokens)
        else:
            child_result = parse_expression(tokens)
    else:
        tokens.unpeek()
        child_result = parse_expression(tokens)
    nl_token = tokens.next_token()
    assert nl_token[0] == NL
    return Tree('statement', [child_result, Tree(nl_token, [])])

def parse_assign(tokens: Tokens) -> Tree:
    identifier = tokens.next_token()
    assert identifier[0] == ID
    eq = tokens.next_token()
    assert eq[0] == EQ
    rhs = parse_expression(tokens)
    return Tree('assign', [Tree(identifier, []), Tree(eq, []), rhs])

def parse_expression(tokens: Tokens) -> Tree:
    atom = parse_atom(tokens)
    token = tokens.peek()
    tokens.unpeek()
    if token[0] in (PLUS, MINUS):
        op = parse_op(tokens)
        rhs = parse_expression(tokens)
        result = Tree('expression', [atom, op, rhs])
    else:
        result = atom
    return result

def parse_atom(tokens: Tokens) -> Tree:
    token = tokens.next_token()
    assert token[0] in (ID, NUM)
    return Tree('atom', [Tree(token, [])])

def parse_op(tokens: Tokens) -> Tree:
    token = tokens.next_token()
    assert token[0] in (PLUS, MINUS)
    return Tree('op', [Tree(token, [])])

def calc_parse(tokens: Token) -> Tree:
    return parse_statements(tokens)

## Step 3: Tie Everything Together

In [4]:
def calc_frontend(source: str) -> Tree:
    return calc_parse(Tokens(calc_lexer(source)))

In [5]:
quick_test_result = calc_frontend('1 + 3 - 5\n')
quick_test_result

Tree(contents='statements', children=[Tree(contents='statement', children=[Tree(contents='expression', children=[Tree(contents='atom', children=[Tree(contents=(4, '1'), children=[])]), Tree(contents='op', children=[Tree(contents=(5, '+'), children=[])]), Tree(contents='expression', children=[Tree(contents='atom', children=[Tree(contents=(4, '3'), children=[])]), Tree(contents='op', children=[Tree(contents=(6, '-'), children=[])]), Tree(contents='atom', children=[Tree(contents=(4, '5'), children=[])])])]), Tree(contents=(1, '\n'), children=[])]), Tree(contents=(0, ''), children=[])])

In [6]:
def tree_to_tuple(tree):
    return tree.contents, [tree_to_tuple(child) for child in tree.children]
tree_to_tuple(quick_test_result)

('statements',
 [('statement',
   [('expression',
     [('atom', [((4, '1'), [])]),
      ('op', [((5, '+'), [])]),
      ('expression',
       [('atom', [((4, '3'), [])]),
        ('op', [((6, '-'), [])]),
        ('atom', [((4, '5'), [])])])]),
    ((1, '\n'), [])]),
  ((0, ''), [])])